In [ ]:
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails
from azureml.opendatasets import OjSalesSimulated
from azureml.automl.core.forecasting_parameters import ForecastingParameters

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [ ]:
ws = Workspace.from_config()

In [ ]:
compute_name = 'compute-cluster'
compute_target = ComputeTarget(ws, compute_name)

In [ ]:
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [ ]:
oj_sales_files = OjSalesSimulated.get_file_dataset()
oj_sales = oj_sales_files.take(10)

In [ ]:
folder_name = "OJ_Sales"
os.makedirs(folder_name, exist_ok=True)

In [ ]:
oj_sales.download(folder_name, overwrite=True)

In [ ]:
OJ_file_path = Path('OJ_Sales').rglob('*.csv')
OJ_files = [x for x in OJ_file_path]
df = pd.concat((pd.read_csv(f) for f in OJ_files))

In [ ]:
df.head(10)

In [ ]:
Dataset.Tabular.register_pandas_dataframe(df, datastore,
                            "OJ Sales Sample")

In [ ]:
experiment_name = 'OJ-Sales-Forecasting'
exp = Experiment(workspace=ws, name=experiment_name)

In [ ]:
dataset_name = "OJ Sales Sample"
dataset = Dataset.get_by_name(ws, dataset_name, version='latest')

In [ ]:
target_column = 'Quantity'

In [ ]:
task = 'forecasting'

In [ ]:
primary_metric = 'normalized_root_mean_squared_error'

In [ ]:
featurization = 'auto'

In [ ]:
params=ForecastingParameters.from_parameters_dict( {'country_or_region_for_holidays':'US',\
                                                    'drop_columns_names':'Revenue',\
                                                    'forecast_horizon': 6,\
                                                    'target_rolling_window_size': 'auto',\
                                                    'target_lags': 'auto',\
                                                    'feature_lags': 'auto',\
                                                    'seasonality': 'auto',\
                                                    'short_series_handling': True,\
                                                    'use_stl': 'season_trend',\
                                                    'time_column_name':'WeekStarting',\
                                                    'time_series_id_column_names':['Store','Brand'],\
                                                    'short_series_handling_configuration': 'auto'},\
                                                    validate_params=True)

In [ ]:
config = AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     n_cross_validations=3,
                     model_explainability=True,
                     enable_stack_ensemble=False,
                     enable_voting_ensemble=True,
                     forecasting_parameters=params)

In [ ]:
AutoML_run = exp.submit(config, show_output = True)
RunDetails(remote_run).show()

In [ ]:
params=ForecastingParameters.from_parameters_dict( {'country_or_region_for_holidays':None,\
                                                    'drop_columns_names':['Revenue','Price','Advert'],\
                                                    'forecast_horizon': 6,\
                                                    'target_rolling_window_size': None,\
                                                    'target_lags': None,\
                                                    'feature_lags': None,\
                                                    'seasonality': 'auto',\
                                                    'short_series_handling': True,\
                                                    'use_stl': 'season_trend',\
                                                    'time_column_name':'WeekStarting',\
                                                    'time_series_id_column_names':['Store','Brand'],
                                                    'short_series_handling_configuration': 'auto'},\
                                                    validate_params=True)

In [ ]:
AutoML_run = exp.submit(config, show_output = True)
RunDetails(remote_run).show()

In [ ]:
description = 'Best AutoML Forecasting Run using OJ Sales Sample Data.' 
tags = {'project' : "OJ Sales", "creator" : "your name"} 
model_name = 'OJ-Sales-Sample-Forecasting-AutoML' 

In [ ]:
AutoML_run.register_model(model_name=model_name, description=description, tags=tags)

In [ ]:
description = 'Best AutoML Forecasting Run using OJ Sales Sample Data.'
tags = {'project' : "OJ Sales", "creator" : "your name", "metric" : "R2 Score"} 
model_name = 'OJ-Sales-Sample-Forecasting-AutoML-R2'
AutoML_run.register_model(model_name=model_name, description=description, tags=tags, metric = 'r2_score')

In [ ]:
experiment_name = 'OJ-Sales-Forecasting'
exp = Experiment(workspace=ws, name=experiment_name) 
AutoML_run = AutoMLRun(experiment = exp, run_id = 'your_run_id’) 
description = 'Retrieved AutoML Forecasting Run for OJ Sales Data.'
tags = {'project' : "OJ Sales", "creator" : "your name"} 
model_name = 'OJ-Sales-Sample-Forecasting-AutoML-Retrieved'
AutoML_run.register_model(model_name=model_name, description=description, tags=tags)